In [ ]:
conn = psycopg2.connect(
    host="fit3164db.cxkhqsoitzhb.ap-southeast-2.rds.amazonaws.com", 
    port=5432,
    user="postgres",
    password="fit3164d13edas",
    database = "testdb"
)
conn.autocommit = True
cursor = conn.cursor()

In [ ]:
cursor.execute("select * from forecasts")
forecast_data = cursor.fetchall()

In [ ]:
forecast

In [ ]:
# converting data types:
def convert_dtypes(df, actual=True):
    if actual:
        numeric_cols = ['month','hour', 'type_of_day', 'load', 'pressure', 'cloud_cover', 'humidity', 'temperature', 'wind_speed', 'wind_direction']
    else:
        numeric_cols = ['month','hour', 'type_of_day', 'pressure', 'cloud_cover', 'temperature', 'wind_speed', 'wind_direction']
    
    date_cols = ['datetime', 'date']
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
        
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    
    return df
actuals = convert_dtypes(actuals)
forecast = convert_dtypes(forecast, False)

In [ ]:
#lags for load 

def get_lag_leads(data, num_lags=168, forward_pred=48): 
    cols = ['pressure', 'cloud_cover', 'temperature', 'wind_speed',
       'wind_direction']

    #for load
    for i in range(1, num_lags + 1):
        data[f'load_lag_{i}'] = data['load'].shift(i) 
    
    for i in range(1, forward_pred+1):    #lead 47 variables
        data[f'load_lead_{i}'] = data['load'].shift(-i)


    #get lead variables
    for c in cols:
        for i in range(1, forward_pred+1):
            data[f'{c}_lag_{i}'] = data[c].shift(-i)

    #for lagged weather variables
    for c in cols:
        for i in range(1, num_lags+1):
            data[f'{c}_lag_{i}'] = data[c].shift(i)

    return data

In [ ]:
actuals[['datetime','load']]

In [ ]:
data = pd.merge(actuals[['datetime','load']], forecast, on='datetime')

encode(data, 'hour', 24)
encode(data, 'month', 12)
encode(data, 'type_of_day', 31)

#call this drop in our model file

# data = data.drop(['id', 'date','month', 'hour', 'type_of_day'], axis=1)

data.set_index
data = data.rename(columns = {'type_of_day_sin': 'day_sin', 
                            'type_of_day_cos': 'day_cos'})

In [ ]:
data = get_lag_leads(data)

In [ ]:
data.tail()

In [ ]:
filename = r"ModelsData.csv"
data.to_csv(filename, index=False, header=True)